# Forecasting time series using VAR

The task is to build a VAR model to predict the sales amounts in different subcategories of furniture. The data is in "Superstore.xlsx", available on Blackboard (the source of the data is [here](https://www.kaggle.com/pruthvi1995/superstore-sales)).

The accuracy of the models should be measured in terms of RMSE and compared to a persistence baseline.

Please complete the solution by writing code and comments in places indicated with "???"

In [ ]:
# setting logging to print only error messages from Sklearnex
import logging
logging.basicConfig()
logging.getLogger("SKLEARNEX").setLevel(logging.ERROR)

import warnings
warnings.filterwarnings("ignore")

import time

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm

from sklearn.metrics import mean_squared_error

# Step 1. Load data

We will select only sales relating to Furniture, and will use only the columns "Order Date", "Category", "Sub-Category", and "Sales".

Note `read_excel` will guess that "Order Date" contains dates and will convert the column to the datetime type.

In [ ]:
df = pd.read_excel("Superstore.xlsx", usecols=[???])
df = df[df['Category'] == 'Furniture']

# once the relevant rows have been selected, delete the Category column
del df["Category"]

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
# Check if there are any missing values
df.isnull().sum()

What are the unique subcategories of furniture?

In [ ]:
df['Sub-Category'].unique()

There are 4 different subcategories. We will create a dataframe for each, transform each separately, and then merge all of them into one dataframe that can be input into the VAR algorithm.

The function `get_sub_category_data` takes the full dataframe, extracts rows corresponding to a given subcategory, calculates daily sales and then groups the daily sales into weekly amounts (in order to smooth out daily fluctuations).

In [ ]:
def get_sub_category_data(df, name):
    d = df[df['Sub-Category'] == name]
    # Obtain daily amounts of sales
    d = d.groupby('Order Date').sum()
    # Group into weekly amounts
    d = d.resample('W').sum()
    return d

Let's check the dataframe produced for the "Chairs" subcategory.

In [ ]:
chairs_df = get_sub_category_data(df, "Chairs")
chairs_df.shape

In [ ]:
type(chairs_df)

In [ ]:
chairs_df.head(3)

Now we can go over the subcategories, produce a dataframe for each, storing them in a list.

In [ ]:
subcat_dfs = []
for x in df['Sub-Category'].unique():
    d = get_sub_category_data(df, x)
    d.columns = [x]
    print(f"Produced a df for {x}, its shape is {d.shape}")
    subcat_dfs.append(d)

All the four dataframes have around 208 rows, which is the number of rows in the original dataset, if it is grouped into weekly amounts of sales. There is, however, 3 weeks with no sales for Tables and 1 week with no sales for Bookcases.

Now, merge them one by one. We will use the intersection, that is, throwing away those rows for which at least one other subcategory has no sales data. So we are effectively deleting rows with missing data.

In [ ]:
# put the first dataframe into df2, the output dataframe

df2 = subcat_dfs[0]
for x in subcat_dfs[1:]:
    # indicate that we merge on the indices of the two dataframes
    df2 = pd.merge(df2, x, how="inner", left_index=True, right_index=True)

df2.shape

This is how the final dataset looks like: we have weekly observations, across the four subcategories of furniture.

In [ ]:
df2

# Step 2. Train-test split

The produced dataframe can now be split into the train and test parts.

In [ ]:
???

# make sure the training and test sets have the same column name as dfs
train_set.columns = df2.columns
test_set.columns = df2.columns

print(f"{train_set.shape[0]} train and {test_set.shape[0]} test instances")

# Step 3. Exploratory Data Analysis

Let's plot the data.

In [ ]:
train_set.plot(figsize=(16,3))

There is quite a bit of volatility within specific subcategories, and there is no apparent relationship between the four time series, so the forecasting problem may be quite hard.

# Step 4. Data cleaning and transformation

Before we can start building a model, we need to ensure the data is **stationary**. We will use the Augmented Dickey-Fuller (ADF) test and the KPSS (Kwiatkowski-Phillips-Schmidt-Shin) tests to test the series for stationarity.

In [ ]:
from statsmodels.tsa.stattools import adfuller, kpss

# test each time series one by one
for x in train_set.columns:
    print(x)
    ???
    print(f"ADF, p-value: {adf_pval}")
    ???
    print(f"KPSS, p-value: {kpss_pval}")
    print()

Comment??? (2-3 sentences)

For consistency, we will difference all the four series.

In [ ]:
???
train_diff.head(3)

In [ ]:
for x in train_diff.columns:
    print(x)
    ???
    print(f"ADF, p-value: {adf_pval:.3f}")
    ???
    print(f"KPSS, p-value: {kpss_pval:.3f}")
    print()

Comment??? (2-3 sentences)

In [ ]:
???
test_diff.head(3)

# Step 5. Build models

## 5.1 Baseline

The persistence baseline is generating the previous day's sales as the prediction for this day.

We'll generate a baseline for each subcategory separately and then calculate their average RMSE.

In [ ]:
baseline_rmse = {}
for x in test_diff.columns:
    baseline_predictions = test_diff[x].shift()[1:]
    mse = ???
    baseline_rmse[x] = np.sqrt(mse)
    print(f"{x}: {baseline_rmse[x]:.3f}")

aver = np.array(list(baseline_rmse.values())).mean()

print(f"\nAverage over the subcategories: {aver:.3f}")

## 5.2 VAR models

The first step is to select the order of VAR, i.e. the optimal number of lags.

In [ ]:
from statsmodels.tsa.vector_ar.var_model import VAR

???

??? (Comment on the choice of the number of lag)

In [ ]:
var_model = ???

In [ ]:
var_model.summary()

??? (2-3 sentences commenting on the constructed model)

# Step 6. Evaluate the model on the test set

We'll use the same evaluation setup as with the ARIMA model: we'll make one-step-ahead forecasts, and we will re-train the model on training data plus any test data that has already been used for evaluation.

In [ ]:
# buffers keeping training + previously seen test data
history = train_diff.values
predictions = []

# for each test observation, take the first 200 for convenience
for i, test_obs in enumerate(test_diff.values):
    
    # build a model using the current buffers
    model = VAR(history).fit(2)
        
    # forecast the value for the test instance, supplying corresponding exogenous variables 
    yhat = model.forecast(model.y, steps=1)
    
    # remember the forecasted value
    predictions.append(yhat[0])
    
    # update the buffers for the endogenous and exogenous variables
    history = np.append(history, test_obs.reshape(1, -1), axis=0)

predictions = pd.DataFrame(predictions, columns=test_diff.columns, index=test_diff.index)

predictions.head(3)

We can plot the predictions:

In [ ]:
???

Finally, let's calculate the RMSE scores within each subcategory and the average RMSE across the subcategories.

In [ ]:
var_rmse = {}
for x in test_diff.columns:
    mse = ???
    var_rmse[x] = np.sqrt(mse)
    print(f"{x} RMSE: {var_rmse[x]:.3f}")

aver = np.array(list(var_rmse.values())).mean()
print(f"Average over subcategories: {aver:.3f}")

Let's compare these results to those achieved by the baseline method. 

In [ ]:
scores_df = pd.DataFrame([baseline_rmse, var_rmse]).transpose()
scores_df.columns = ["Baseline", "VAR"]

# add a column showing error rate reduction compared to the baseline
scores_df["% reduction"] = 100*(scores_df["Baseline"]-scores_df["VAR"])/scores_df["Baseline"]

scores_df

In [ ]:
scores_df["% reduction"].mean()

# Conclusion

??? (one-two sentences on the quality of the constructed VAR model)